# Inverse static (captive test)

In [ ]:
# %load imports.py
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl

from src.symbols import *
from src.parameters import *
import src.symbols as symbols
from src import prime_system
from src.models import regression
from src.visualization.regression import show_pred
from src.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import src.models.vmm_martin  as vmm 
from src.models.vmm import ModelSimulator
from example_ship2 import ship_parameters, df_parameters


if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows
    
from src.visualization.plot import track_plot, plot

In [ ]:
from src.models.regression import ForceRegression

In [ ]:
df_VCT = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT.head()

In [ ]:
interesting = ['u','v','r','delta','thrust','fx','fy','mz']
data = df_VCT[interesting].copy()

In [ ]:
added_masses = df_parameters.groupby(by='state').get_group('dot')['prime'].dropna().to_dict()

In [ ]:
regression = ForceRegression(vmm=vmm, data=data, added_masses=added_masses, ship_parameters=ship_parameters)

In [ ]:
regression.show()

In [ ]:
regression.parameters

In [ ]:
model = regression.create_model(control_keys=['delta','thrust'])

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]
df['x0']-=df.iloc[0]['x0']
df['y0']-=df.iloc[0]['y0']
df['thrust'] = df[['Prop/PS/Thrust','Prop/SB/Thrust']].sum(axis=1)
